In [1]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
import numpy as np
import pymysql

In [2]:
#conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='101Monkeybob!',db='Final_Project')
#df=pd.read_sql_query("SELECT * FROM 'house' ",conn)


In [3]:
# Import Clean House Data
house_data = pd.read_csv("clean_house_data_ML.csv",low_memory = False)
imp_df = pd.DataFrame(house_data)
imp_df.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,PRICE,INDEX_BIN
0,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,1325000.0,52
1,1,0,6,3,2006.0,1.0,0,2090,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0,33232.0,1
2,2,1,5,3,2012.0,1.0,1,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,907400.0,36
3,2,1,8,3,2012.0,1.0,2,2090,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,1065000.0,42
4,2,1,7,2,2009.0,1.0,1,838,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0,216000.0,8


In [4]:
imp_df=imp_df.drop("INDEX_BIN", axis = 1)

In [5]:
index = imp_df[imp_df.PRICE >= 1500000].index
index2 = imp_df[imp_df.PRICE <= 100000].index
imp_df = imp_df.drop(index)
imp_df = imp_df.drop(index2)

In [6]:
# Labels are the values we want to predict
labels = np.array(imp_df['PRICE'])

# Remove the labels from the features
# axis 1 refers to the columns
imp_df= imp_df.drop('PRICE', axis = 1)

# Saving feature names for later use
feature_list = list(imp_df.columns)

# Convert to numpy array
features = np.array(imp_df)

In [7]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [8]:
# Resample the training data with the BalancedRandomForestClassifier
model = RandomForestRegressor(n_estimators = 1000, random_state = 2)
# Train the model on training data
model.fit(train_features, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=2)

In [9]:
# Use the forest's predict method on the test data
predictions = model.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 220514.12 degrees.


In [10]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 35.97 %.


In [22]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

fig = plt.figure(figsize=(15, 10))
plot_tree(model.estimators_[0], 
          feature_names=feature_list,
          class_names=labels, 
          filled=True, impurity=True, 
          rounded=True)

[Text(489.0678835459922, 536.4473684210526, 'QUADRANT_NW <= 0.5\nsquared_error = 93876704646.236\nsamples = 14655\nvalue = 534743.579'),
 Text(275.6865665875174, 522.1421052631579, 'ZIPCODE <= 20017.5\nsquared_error = 62006991139.641\nsamples = 8856\nvalue = 441080.121'),
 Text(135.31534381716392, 507.8368421052632, 'WARD <= 5.5\nsquared_error = 74695994888.607\nsamples = 4251\nvalue = 519916.318'),
 Text(56.702456784229575, 493.5315789473684, 'ZIPCODE <= 20006.5\nsquared_error = 54938055496.022\nsamples = 2002\nvalue = 451135.888'),
 Text(30.41331860132491, 479.2263157894737, 'LANDAREA <= 789.5\nsquared_error = 43987947608.752\nsamples = 741\nvalue = 428621.418'),
 Text(30.35342475977125, 464.92105263157896, 'squared_error = 0.0\nsamples = 1\nvalue = 1050000.0'),
 Text(30.47321244287857, 464.92105263157896, 'LANDAREA <= 1648.5\nsquared_error = 43419516909.218\nsamples = 740\nvalue = 427590.939'),
 Text(24.884079800891243, 450.61578947368423, 'LANDAREA <= 1646.5\nsquared_error = 398665

Error in callback <function flush_figures at 0x0000017F0CFA7DC8> (for post_execute):


KeyboardInterrupt: 

In [ ]:
fig.savefig('Final_Project\picturesfigure_name.png')